In [1]:
import glob
import pandas as pd

from os.path import basename
import pickle

from level2.level2_utils import *
from level2.level2_roi_extraction import get_roi_data_for_l2_betas
#import modules from files in a parallel directory "direct_regression"

from direct_regression.get_all_series import get_beta_img, get_roi_data, get_moment_trial_type_revealed, get_behavioral_data_with_moment_trial_type_revealed, mask_3d_subject_image
from direct_regression.get_all_series import get_all_subj_df
from direct_regression.fmri_utils import *



In [2]:

config = load_config("direct_regression/config.yml")
analysis_name = 'conditions'


#get the masks
mask_locations = [
    #config['mask_location'] + 'failure_related/',
    config['mask_location'] + 'aim3/neurosynth/',
    config['mask_location'] + 'aim3/'

    #config['mask_location'] + 'failure_related/harvardoxford/',
    #config['mask_location'] + 'striatum/'
]
mask_df = get_mask_df_from_mask_locations(mask_locations)
#get the list of raw nii files
glob_path = config['fmriprep_dir'] + config['nii_raw_path']

Benjamins-MacBook-Pro-2.local
{'dev_scripts_path': '/Users/benjaminsmith/Google Drive/oregon/code/DEV_scripts/', 'rural_data_dir': '/Users/benjaminsmith/Dropbox (University of Oregon)/UO-SAN Lab/Berkman Lab/Devaluation/analysis_files/data/', 'rubric_dir': '/Users/benjaminsmith/Dropbox (University of Oregon)/UO-SAN Lab/Berkman Lab/Devaluation/DEV_scoring_rubrics', 'dev_analysis_data_dir': '/Users/benjaminsmith/Dropbox (University of Oregon)/UO-SAN Lab/Berkman Lab/Devaluation/analysis_files/data/', 'input_mat_path': '/Users/benjaminsmith/oregon/code/DEV_scripts/fMRI/fx/multiconds/SST/full_duration/conditions', 'dropbox_data_dir': '/Users/benjaminsmith/Dropbox (University of Oregon)/UO-SAN Lab/Berkman Lab/Devaluation/analysis_files/data/', 'fmriprep_dir': '/Users/benjaminsmith/Google Drive/oregon/data/DEV/bids_data/derivatives/fmriprep_2022/', 'nii_raw_path': 'sub-DEV*/ses-wave*/func/s6_sub-DEV*_ses-wave*_task-SST_acq-1_space-MNI152NLin2009cAsym_desc-preproc_bold.nii', 'mask_location': '/

In [ ]:
## SST
train_betas_with_data = get_data_for_confirmed_train_subjs(
    beta_glob = config['nonbids_data_path'] + "fMRI/fx/models/SST/wave1/conditions/sub-DEV*/",
    nonbids_data_path = config['nonbids_data_path'],
    #ml_data_folderpath = ml_data_folderpath,
    ml_scripting_path = config['dev_scripts_path'] + "/fMRI/ml",
    dropbox_datapath=config['dropbox_data_dir'],
    exclude_test_subjs=False
)
train_betas_with_data['wave']=1

#we're not interestd in getting contrasts; comment this out.
#betas_with_contrasts = get_contrasts_for_betas(train_betas_with_data)
betas_with_paths = get_beta_fnames_for_beta_dirs(train_betas_with_data)

beta_name_list = [
    'CorrectGo',
    'CorrectStop',
    'FailedStop',
    #'Cue',
    #'FailedGo'
    ]

#get the ROI data
roi_data = get_roi_data_for_l2_betas(betas_with_paths, beta_name_list, mask_df)

roi_data.to_csv(config['dropbox_data_dir'] + '/subject_sst_avg_roi_data_raw.csv')



In [3]:
if False:
    ## WTP
    train_betas_with_data = get_data_for_confirmed_train_subjs(
        beta_glob = config['nonbids_data_path'] + "fMRI/fx/models/WTP/wave1/conditions_liked_disliked/sub-DEV*/",
        nonbids_data_path = config['nonbids_data_path'],
        #ml_data_folderpath = ml_data_folderpath,
        ml_scripting_path = config['dev_scripts_path'] + "/fMRI/ml",
        dropbox_datapath=config['dropbox_data_dir'],
        exclude_test_subjs=False,
        task='WTP'
    ).copy()
    train_betas_with_data['wave']=1

loaded 275 rows from data_by_ppt.csv
246 subjects with no reported problems in task Redcap scanner notes, added to the provision useable_dev_id list.
3 subjects excludeable for missing scan data.
243 subjects remaining on the provision useable_dev_id list from the redcap list after excluding subjects with missing scan data.
243 subjects remaining on the provisional useable_dev_id list from the redcap list after excluding subjects excluded by motion quality process.
beta paths before exclusion: 278
beta paths after exclusion via nsc_subject_exclusions and the provision useable_dev_id list: 239


/Users/benjaminsmith/Google Drive/oregon/code/DEV_scripts/fMRI/fx/models/SST/level2/level2_utils.py:188: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  useable_train_betas_with_data.sort_values('subject_id', inplace=True)


In [5]:
train_betas_with_data[train_betas_with_data.subject_id=='DEV198']

,subject_id,spm_l2_path,spm_l2_path_description,wave


In [6]:

#here we're going to find a way to pass betas themselves instead of contrasts
def get_beta_fnames_for_beta_dirs(
        beta_dir_df
):
    beta_dir_df['spm_l2_targeted_contrast_filepath'] = None

    beta_dir_df.reset_index(inplace=True,drop=True)

    for beta_i, beta_folderpath in enumerate(beta_dir_df.spm_l2_path):
        print(beta_folderpath)
        # load the mat file associated with the current run
        subj_l1_mat_config = scipy.io.loadmat(str(
            beta_folderpath + '/SPM.mat'), appendmat=False, simplify_cells=True, variable_names='SPM')

        # get the list of level 1 contrasts that are included in the level 1 analysis we're examining
        # contrasts_in_run = [contrast['name'] for contrast in subj_l1_mat_config['SPM']['xCon']]

        # which is it?
        # contrast_index = [c_i for c_i, contrast in enumerate(subj_l1_mat_config['SPM']['xCon'])
        #                   if contrast['name'] == contrast_to_extract
        #                   ][0]

        for b_i, vbeta in enumerate(subj_l1_mat_config['SPM']['Vbeta']):
            beta_readable_description_wrapped = re.search(r'Sn\(\d\) (.*?)(\^\d)?(\*bf\(1\))?$', vbeta['descrip'])
            if beta_readable_description_wrapped is None:
                raise Exception("could not find beta description in " + vbeta['descrip'])
                
            beta_readable_description = beta_readable_description_wrapped[1]
            #print(str(b_i) + ": " + beta_readable_description + " " + vbeta['fname'])
            beta_dir_df.loc[beta_i, 'beta_' + beta_readable_description + "_fname"] =vbeta['fname']

        # if contrast_index is None:
        #     print("this run does not contain the targeted contrast.")
        # else:
        #     return (contrast_index)

        #betas.loc[beta_i, 'spm_l2_targeted_contrast_filepath'] =
    # test if the contrast we want is there

    return (beta_dir_df)



In [7]:


#we're not interestd in getting contrasts; comment this out.
#betas_with_contrasts = get_contrasts_for_betas(train_betas_with_data)
betas_with_paths = get_beta_fnames_for_beta_dirs(train_betas_with_data)



/Users/benjaminsmith/Google Drive/oregon/data/DEV/nonbids_data/fMRI/fx/models/WTP/wave1/conditions_liked_disliked/sub-DEV005/
/Users/benjaminsmith/Google Drive/oregon/data/DEV/nonbids_data/fMRI/fx/models/WTP/wave1/conditions_liked_disliked/sub-DEV006/
/Users/benjaminsmith/Google Drive/oregon/data/DEV/nonbids_data/fMRI/fx/models/WTP/wave1/conditions_liked_disliked/sub-DEV008/
/Users/benjaminsmith/Google Drive/oregon/data/DEV/nonbids_data/fMRI/fx/models/WTP/wave1/conditions_liked_disliked/sub-DEV010/
/Users/benjaminsmith/Google Drive/oregon/data/DEV/nonbids_data/fMRI/fx/models/WTP/wave1/conditions_liked_disliked/sub-DEV012/
/Users/benjaminsmith/Google Drive/oregon/data/DEV/nonbids_data/fMRI/fx/models/WTP/wave1/conditions_liked_disliked/sub-DEV013/
/Users/benjaminsmith/Google Drive/oregon/data/DEV/nonbids_data/fMRI/fx/models/WTP/wave1/conditions_liked_disliked/sub-DEV014/
/Users/benjaminsmith/Google Drive/oregon/data/DEV/nonbids_data/fMRI/fx/models/WTP/wave1/conditions_liked_disliked/sub-

FileNotFoundError: [Errno 2] No such file or directory: '/Users/benjaminsmith/Google Drive/oregon/data/DEV/nonbids_data/fMRI/fx/models/WTP/wave1/conditions_liked_disliked/sub-DEV222//SPM.mat'

In [ ]:
beta_name_list = [
    'liked',
    'disliked'
    ]

#get the ROI data
roi_data = get_roi_data_for_l2_betas(betas_with_paths, beta_name_list, mask_df)

roi_data.to_csv(config['dropbox_data_dir'] + '/subject_sst_avg_roi_data_raw.csv')


In [ ]:
## ROC
raise NotImplementedError("ROC not implemented yet")

In [ ]:
#save
with open(config['dropbox_data_dir'] + '/subject_sst_avg_roi_data_raw.pkl', 'wb') as handle:
    pickle.dump(roi_data, handle, protocol=pickle.HIGHEST_PROTOCOL)



,subject_id,spm_l2_path,spm_l2_path_description,wave,spm_l2_targeted_contrast_filepath,beta_CorrectGo_fname,beta_CorrectStop_fname,beta_FailedStop_fname,beta_Cue_fname,beta_FailedGo_fname,beta_R1_fname,beta_R2_fname,beta_R3_fname,beta_R4_fname,beta_R5_fname,beta_constant_fname
0,DEV006,/Users/benjaminsmith/Google Drive/oregon/data/...,'/Users/benjaminsmith/Google Drive/oregon/data...,1,None,beta_0001.nii,beta_0002.nii,beta_0003.nii,beta_0004.nii,beta_0005.nii,beta_0006.nii,beta_0007.nii,beta_0008.nii,beta_0009.nii,beta_0010.nii,beta_0011.nii
1,DEV008,/Users/benjaminsmith/Google Drive/oregon/data/...,'/Users/benjaminsmith/Google Drive/oregon/data...,1,None,beta_0001.nii,beta_0002.nii,beta_0003.nii,beta_0004.nii,NaN,beta_0005.nii,beta_0006.nii,beta_0007.nii,beta_0008.nii,beta_0009.nii,beta_0010.nii
2,DEV013,/Users/benjaminsmith/Google Drive/oregon/data/...,'/Users/benjaminsmith/Google Drive/oregon/data...,1,None,beta_0001.nii,beta_0002.nii,beta_0003.nii,beta_0004.nii,beta_0005.nii,beta_0006.nii,beta_0007.nii,beta_0008.nii,beta_0009.nii,beta_0010.nii,beta_0011.nii
3,DEV014,/Users/benjaminsmith/Google Drive/oregon/data/...,'/Users/benjaminsmith/Google Drive/oregon/data...,1,None,beta_0001.nii,beta_0002.nii,beta_0003.nii,beta_0004.nii,NaN,beta_0005.nii,beta_0006.nii,beta_0007.nii,beta_0008.nii,beta_0009.nii,beta_0010.nii
4,DEV015,/Users/benjaminsmith/Google Drive/oregon/data/...,'/Users/benjaminsmith/Google Drive/oregon/data...,1,None,beta_0001.nii,beta_0002.nii,beta_0003.nii,beta_0004.nii,beta_0005.nii,beta_0006.nii,beta_0007.nii,beta_0008.nii,beta_0009.nii,beta_0010.nii,beta_0011.nii
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
204,DEV307,/Users/benjaminsmith/Google Drive/oregon/data/...,'/Users/benjaminsmith/Google Drive/oregon/data...,1,None,beta_0001.nii,beta_0002.nii,beta_0003.nii,beta_0004.nii,beta_0005.nii,beta_0006.nii,beta_0007.nii,beta_0008.nii,beta_0009.nii,beta_0010.nii,beta_0011.nii
205,DEV308,/Users/benjaminsmith/Google Drive/oregon/data/...,'/Users/benjaminsmith/Google Drive/oregon/data...,1,None,beta_0001.nii,beta_0002.nii,beta_0003.nii,beta_0004.nii,NaN,beta_0005.nii,beta_0006.nii,beta_0007.nii,beta_0008.nii,beta_0009.nii,beta_0010.nii
206,DEV309,/Users/benjaminsmith/Google Drive/oregon/data/...,'/Users/benjaminsmith/Google Drive/oregon/data...,1,None,beta_0001.nii,beta_0002.nii,beta_0003.nii,beta_0004.nii,NaN,beta_0005.nii,beta_0006.nii,beta_0007.nii,beta_0008.nii,beta_0009.nii,beta_0010.nii
207,DEV310,/Users/benjaminsmith/Google Drive/oregon/data/...,'/Users/benjaminsmith/Google Drive/oregon/data...,1,None,beta_0001.nii,beta_0002.nii,beta_0003.nii,beta_0004.nii,NaN,beta_0005.nii,beta_0006.nii,beta_0007.nii,beta_0008.nii,beta_0009.nii,beta_0010.nii
